In [1]:
# -*- coding: utf-8 -*-
"""Welcome To Colab

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/notebooks/intro.ipynb
"""

!pip install sentence-transformers faiss-cpu

from google.colab import files
uploaded = files.upload()

import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# === Load Extracted Text ===
with open("mosdac_html_text_data.json", "r", encoding="utf-8") as f:
    html_data = json.load(f)

# === Chunking ===
chunk_size = 750
chunks = []
for entry in html_data:
    words = entry["text"].split()
    for i in range(0, len(words), chunk_size):
        chunk_text = " ".join(words[i:i + chunk_size])
        chunks.append({
            "text": chunk_text,
            "source": entry["file"]
        })

print("Chunks:", len(chunks))

# === Embeddings ===
model = SentenceTransformer("all-MiniLM-L6-v2")
texts = [chunk["text"] for chunk in chunks]
embeddings = model.encode(texts, show_progress_bar=True)

# === FAISS Index ===
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings))
print("Index ready.")

# === Semantic Search Function ===
def ask(query, top_k=5):
    q_emb = model.encode([query])
    D, I = index.search(np.array(q_emb), top_k)
    return [{"text": chunks[i]["text"], "source": chunks[i]["source"]} for i in I[0]]

# === Example Use ===
query = "What is the role of INSAT-3DR?"
results = ask(query)
for i, res in enumerate(results, 1):
    print(f"\n--- Result {i} (From: {res['source']}) ---")
    print(res["text"][:500], "...")

!pip install streamlit
!streamlit run app.py

!pip install gradio sentence-transformers faiss-cpu

import json
import faiss
import numpy as np
import gradio as gr
from sentence_transformers import SentenceTransformer

# === Load Extracted Text ===
with open("mosdac_html_text_data.json", "r", encoding="utf-8") as f:
    html_data = json.load(f)

# === Chunking ===
chunk_size = 750
chunks = []
for entry in html_data:
    words = entry["text"].split()
    for i in range(0, len(words), chunk_size):
        chunk_text = " ".join(words[i:i + chunk_size])
        chunks.append({"text": chunk_text, "source": entry["file"]})

# === Embedding Model ===
model = SentenceTransformer("all-MiniLM-L6-v2")
texts = [chunk["text"] for chunk in chunks]
embeddings = model.encode(texts, show_progress_bar=True)

# === FAISS Index ===
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings))

# === Query Function ===
def query_rag(user_input, top_k=3):
    q_emb = model.encode([user_input])
    D, I = index.search(np.array(q_emb), top_k)
    results = []
    for i in I[0]:
        source = chunks[i]["source"]
        preview = chunks[i]["text"][:1000] + ("..." if len(chunks[i]["text"]) > 1000 else "")
        results.append(f"📄 **Source:** `{source}`\n\n{preview}")
    return "\n\n---\n\n".join(results)

# === Gradio Interface ===
interface = gr.Interface(
    fn=query_rag,
    inputs=gr.Textbox(lines=2, placeholder="Ask about ISRO or MOSDAC data..."),
    outputs="markdown",
    title="🔍 ISRO Help Bot (MOSDAC RAG)",
    description="Ask questions based on data crawled from the MOSDAC portal."
)

interface.launch()

!pip install transformers accelerate bitsandbytes

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"  # You can also use "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

Saving mosdac_html_text_data.json to mosdac_html_text_data.json
Chunks: 138


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Index ready.

--- Result 1 (From: /mnt/data/mosdac_extracted/mosdac_complete_crawl4/pages/internal/depth_1/page_0037/page.html) ---
INSAT3R Intense Heavy Rain Over India INSAT3R Intense Heavy Rain Over India Rainfall (mm): GMT Current HeavyRain Rainfall Rainfall (mm) Base Layers Selection:- Single Multiple DEM Bhuvan Maps Open Street Map Natural Earth Black Marble True Marble Previous Forecast/Events Select Date: Tools Analysis Legends District View Area View District View Area View DISTRICTS HAVING HIGH RAIN(>5mm/h) District Name Met-subdivision Rain(mm/h) AREA HAVING HIGH RAIN(>10mm/h) Area(Km*2) Highest Rain(mm/h) Lat/Lon ...

--- Result 2 (From: /mnt/data/mosdac_extracted/mosdac_complete_crawl4/pages/internal/depth_2/page_0006/page.html) ---
Meteorological & Oceanographic Satellite Data Archival Centre Skip to main Content - A A + A A A हिन्दी English Secondary menu SignUp Login Logout Meteorological & Oceanographic Satellite Data Archival Centre Space Applications Centre, ISRO Hom

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://183c24c5071df5f2f1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 13.5 MB/s eta 0:00:00


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct.
401 Client Error. (Request ID: Root=1-685d3cd7-5ebfc50e1a1a62e40b8dc162;44770766-424e-4f18-8b9a-7d45b7c0da02)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.